In [2]:
import requests 
import sys
import json
import time
import math
import signal
import numpy as np

In [3]:
def signal_handler(sig, frame):
    global run
    print('You pressed Ctrl+C!')
    run = False

In [4]:
def rotate( vec, deg ):
    rad = 2 * math.pi / 360 * deg

    out = [0,0]

    out[0] = ( math.cos( rad ) * vec[0] - math.sin( rad ) * vec[1] )
    out[1] = ( math.sin( rad ) * vec[0] + math.cos( rad ) * vec[1] )

    return out

In [5]:
def set_vel(vel):
    OMNIDRIVE_URL = "http://" + ROBOTINOIP + "/data/omnidrive"
    r = requests.post(url = OMNIDRIVE_URL, params = PARAMS, json = vel )
    if r.status_code != requests.codes.ok:
        #print("Error: post to %s with params %s failed", OMNIDRIVE_URL, PARAMS)
        raise RuntimeError("Error: post to %s with params %s failed", OMNIDRIVE_URL, PARAMS)

In [6]:
def bumper():
    BUMPER_URL = "http://" + ROBOTINOIP + "/data/bumper"
    r = requests.get(url = BUMPER_URL, params = PARAMS)
    if r.status_code == requests.codes.ok:
        data = r.json()
        return data["value"]
    else:
        raise RuntimeError("Error: get from %s with params %s failed", BUMPER_URL, PARAMS)

In [7]:
def distances():
    DISTANCES_URL = "http://" + ROBOTINOIP + "/data/distancesensorarray"
    r = requests.get(url = DISTANCES_URL, params = PARAMS)
    if r.status_code == requests.codes.ok:
        data = r.json()
        return data
    else:
        raise RuntimeError("Error: get from %s with params %s failed", DISTANCES_URL, PARAMS)

In [8]:
def OdometryRead():
    ODOMETRY_URL = "http://" + ROBOTINOIP + "/data/odometry"
    r = requests.get(url = ODOMETRY_URL, params = PARAMS)
    if r.status_code == requests.codes.ok:
        data = r.json()
        return data
    else:
        raise RuntimeError("Error: get from %s with params %s failed", ODOMETRY_URL, PARAMS)

In [9]:
class PID:
    def __init__(self,kP,uAboveLimit,uUnderLimit):
        self.kP = kP
        self.uAboveLimit = uAboveLimit
        self.uUnderLimit = uUnderLimit
        
    def PidCal(self,Target,current):
        err = Target - current
        uP = self.kP * err
        u = uP
        
        if(u>self.uAboveLimit):
            u=self.uAboveLimit
        elif(u<self.uUnderLimit):
            u=self.uUnderLimit 
        else:
            u = u
        return u

In [38]:
# helper functions
def pt_to_pt_distance (pt1,pt2):
    distance = np.sqrt((pt2[0] - pt1[0])**2 + (pt2[1] - pt1[1])**2)
    return distance

# returns -1 if num is negative, 1 otherwise
def sgn (num):
  if num >= 0:
    return 1
  else:
    return -1
# Hàm sau nếu phát hiện hai biến bằng nhau thì xuất ra 
# Tại trong quá trình tính toán của MCU có thể xuất ra những giá trị xấp sỉ 
def equalCompare(Value,ComValue):
    if(abs(Value - ComValue) < 0.0001):
        return 1
    else :
        return 0

# path1 = np.array([[0,0],[0.5,0],[1,0.5],[2,-0.2],[3,0],[0,0]])
path1 = np.array([[0,0],[0.5,0.5],[0,1],[-1,0],[0,-1],[0.5,-0.5],[0,0]])
#path1 = [[0.0, 0.0], [0.571194595265405, -0.4277145118491421], [1.1417537280142898, -0.8531042347260006], [1.7098876452457967, -1.2696346390611464], [2.2705328851607995, -1.6588899151216996], [2.8121159420106827, -1.9791445882187304], [3.314589274316711, -2.159795566252656], [3.7538316863009027, -2.1224619985315876], [4.112485112342358, -1.8323249172947023], [4.383456805594431, -1.3292669972090994], [4.557386228943757, -0.6928302521681386], [4.617455513800438, 0.00274597627737883], [4.55408382321606, 0.6984486966257434], [4.376054025556597, 1.3330664239172116], [4.096280073621794, 1.827159263675668], [3.719737492364894, 2.097949296701878], [3.25277928312066, 2.108933125822431], [2.7154386886417314, 1.9004760368018616], [2.1347012144725985, 1.552342808106984], [1.5324590525923942, 1.134035376721349], [0.9214084611203568, 0.6867933269918683], [0.30732366808208345, 0.22955002391894264], [-0.3075127599907512, -0.2301742560363831], [-0.9218413719658775, -0.6882173194028102], [-1.5334674079795052, -1.1373288016589413], [-2.1365993767877467, -1.5584414896876835], [-2.7180981380280307, -1.9086314914221845], [-3.2552809639439704, -2.1153141204181285], [-3.721102967810494, -2.0979137913841046], [-4.096907306768644, -1.8206318841755131], [-4.377088212533404, -1.324440752295139], [-4.555249804461285, -0.6910016662308593], [-4.617336323713965, 0.003734984720118972], [-4.555948690867849, 0.7001491248072772], [-4.382109193278264, 1.3376838311365633], [-4.111620918085742, 1.8386823176628544], [-3.7524648889185794, 2.1224985058331005], [-3.3123191098095615, 2.153588702898333], [-2.80975246649598, 1.9712114570096653], [-2.268856462266256, 1.652958931009528], [-1.709001159778989, 1.2664395490411673], [-1.1413833971013372, 0.8517589252820573], [-0.5710732645795573, 0.4272721367616211], [0, 0], [0.571194595265405, -0.4277145118491421]]
# path1 = np.array([[0,0],[0.5,0],[1,1],[1.5,0.5],[1,0],[-1.5,0.5],[1,-1],[0.5,0],[0,0]])

currentPos = [0, 0]
currentHeading = 330
lastFoundIndex = 0
lookAheadDis = 0.35
pathlen = path1.shape[0];

def pure_pursuit_step (path, currentPos, lookAheadDis, LFindex) :
    global lastFoundIndex
    #Đầu tiên là chúng ta sẽ lấy giá trị hiện tại của robot về để tính toán 
    currentX = currentPos[0]
    currentY = currentPos[1]
    #Cập nhật các biến lastIndex và startIndex để robot biết đã qua những đoạn đường nào 
    #Biến intersection 
    lastFoundIndex = LFindex
    intersection = False
    startingIndex = lastFoundIndex

    for i in range (startingIndex,len(path) - 1):
        # Các điểm 1 2 lần lượt là các điểm ở trong mảng path sẽ tạo ra các đường thẳng 
        x1 = path[i][0] - currentX
        y1 = path[i][1] - currentY
        x2 = path[i+1][0] - currentX
        y2 = path[i+1][1] - currentY
        # Các công thức tính ra khoảng cách giữa hai điểm
        # Và công thức xác định xem có cắt đường tròn không
        dx = x2 - x1
        dy = y2 - y1
        dr = math.sqrt (dx**2 + dy**2)
        D = x1*y2 - x2*y1
        # discriminant để xác định xem có cắt vào đường tròn hay không
        # discriminant > 0 có cắt 
        # discriminant < 0 không cắt
        # discriminant = 0 tiếp tuyến
        discriminant = (lookAheadDis**2) * (dr**2) - D**2

        # Đoạn code tìm điểm cắt 
        if discriminant >= 0:
            # Tìm các điểm cắt trên đường thẳng
            sol_x1 = (D * dy + sgn(dy) * dx * np.sqrt(discriminant)) / dr**2
            sol_x2 = (D * dy - sgn(dy) * dx * np.sqrt(discriminant)) / dr**2
            sol_y1 = (- D * dx + abs(dy) * np.sqrt(discriminant)) / dr**2
            sol_y2 = (- D * dx - abs(dy) * np.sqrt(discriminant)) / dr**2

            # Đưa các điểm tìm được vào mảng và đưa nó về toạ độ thực bằng cách cộng vào các trục thực
            sol_pt1 = [sol_x1 + currentX, sol_y1 + currentY]
            sol_pt2 = [sol_x2 + currentX, sol_y2 + currentY]

            #Xác định các giá trị X Y mút giữa hai điểm trên quỷ đạo để xác định xem điểm cắt có nằm trong khoảng đó hay không
            minX = min(path[i][0], path[i+1][0])
            minY = min(path[i][1], path[i+1][1])
            maxX = max(path[i][0], path[i+1][0])
            maxY = max(path[i][1], path[i+1][1])

            # Xác định xem có điểm nào trong khoảng giữa hai điểm hay không
            # Nếu có thì kích biến intersectFound lên 1
            if (
                (minX <= sol_pt1[0] <= maxX and minY <= sol_pt1[1] <= maxY) 
                or (equalCompare(sol_pt1[0],minX) == 1 or equalCompare(sol_pt1[0],maxX) == 1 
                or equalCompare(sol_pt1[1],minY) == 1 or equalCompare(sol_pt1[1],maxY) == 1) 
                or (minX <= sol_pt2[0] <= maxX and minY <= sol_pt2[1] <= maxY) 
                or (equalCompare(sol_pt2[0],minX) == 1 or equalCompare(sol_pt2[0],maxX) == 1 
                or equalCompare(sol_pt2[1],minY) == 1 or equalCompare(sol_pt2[1],maxY) == 1)) :
                intersectFound = True 

                # TH 1 : Nếu cả hai điểm ở trong đều ở trong khoảng điểm :
                if (
                (minX <= sol_pt1[0] <= maxX and minY <= sol_pt1[1] <= maxY) 
                or (equalCompare(sol_pt1[0],minX) == 1 or equalCompare(sol_pt1[0],maxX) == 1 
                or equalCompare(sol_pt1[1],minY) == 1 or equalCompare(sol_pt1[1],maxY) == 1) 
                and (minX <= sol_pt2[0] <= maxX and minY <= sol_pt2[1] <= maxY) 
                or (equalCompare(sol_pt2[0],minX) == 1 or equalCompare(sol_pt2[0],maxX) == 1 
                or equalCompare(sol_pt2[1],minY) == 1 or equalCompare(sol_pt2[1],maxY) == 1)) :# Chúng ta sẽ kiểm tra xem điểm nào gần điểm tiếp theo cần đến trong quỷ đạo hơn :
                    if pt_to_pt_distance(sol_pt1, path[i+1]) < pt_to_pt_distance(sol_pt2, path[i+1]):
                        goalPt = sol_pt1
                    else:
                        goalPt = sol_pt2

                # TH2 : Nếu chỉ có 1 trong hai điểm nằm trong khoảng :
                else :
                    if (
                    (minX <= sol_pt1[0] <= maxX and minY <= sol_pt1[1] <= maxY) 
                    or (equalCompare(sol_pt1[0],minX) == 1 or equalCompare(sol_pt1[0],maxX) == 1 
                    or equalCompare(sol_pt1[1],minY) == 1 or equalCompare(sol_pt1[1],maxY) == 1) ):
                        goalPt = sol_pt1
                    else:
                        goalPt = sol_pt2
                
                # Tính xem là khoảng cách của setpoint tới điểm tiếp theo xem có gần hơn với vị trí robot hay không :
                # Thoát giữ index để biết là hiện tại đang ở đoạn nào 
                
                if pt_to_pt_distance (goalPt, path[i+1]) < pt_to_pt_distance ([currentX, currentY], path[i+1]):
                    lastFoundIndex = i
                    break
                else :
                    # if lastFoundIndex == path1.shape(1)-2:
                    #     goalPt = [path[lastFoundIndex+1][0], path[lastFoundIndex+1][1]]
                    # Trường hợp là vị trí setpoint xa hơn vị trí robot:
                    # index + lên một để set đến đoạn đường tiếp theo và tránh việc robot đi ngược lại
                    lastFoundIndex = i+1
            # Nếu không có điểm nào cắt :
            else :
                foundIntersection = False
                goalPt = [path[lastFoundIndex][0], path[lastFoundIndex][1]]
    
    return goalPt

In [72]:
# helper functions
def pt_to_pt_distance (pt1,pt2):
    distance = np.sqrt((pt2[0] - pt1[0])**2 + (pt2[1] - pt1[1])**2)
    return distance

# returns -1 if num is negative, 1 otherwise
def sgn (num):
  if num >= 0:
    return 1
  else:
    return -1
# Hàm sau nếu phát hiện hai biến bằng nhau thì xuất ra 
# Tại trong quá trình tính toán của MCU có thể xuất ra những giá trị xấp sỉ 
def equalCompare(Value,ComValue):
    if(abs(Value - ComValue) < 0.0001):
        return 1
    else :
        return 0
        
# Khai báo các biến cho Bộ Bám đường thuần tuý 
currentPos = [0, 0]
currentHeading = 330
lastFoundIndex = 0
lookAheadDis = 0.35
pathlen = path1.shape[0];

def pure_pursuit_step (path, currentPos, lookAheadDis, LFindex) :
    global lastFoundIndex
    #Đầu tiên là chúng ta sẽ lấy giá trị hiện tại của robot về để tính toán 
    currentX = currentPos[0]
    currentY = currentPos[1]
    #Cập nhật các biến lastIndex và startIndex để robot biết đã qua những đoạn đường nào 
    #Biến intersection 
    lastFoundIndex = LFindex
    intersection = False
    startingIndex = lastFoundIndex

    for i in range (startingIndex,len(path) - 1):
        # Các điểm 1 2 lần lượt là các điểm ở trong mảng path sẽ tạo ra các đường thẳng 
        x1 = path[i][0] - currentX
        y1 = path[i][1] - currentY
        x2 = path[i+1][0] - currentX
        y2 = path[i+1][1] - currentY
        # Các công thức tính ra khoảng cách giữa hai điểm
        # Và công thức xác định xem có cắt đường tròn không
        dx = x2 - x1
        dy = y2 - y1
        dr = math.sqrt (dx**2 + dy**2)
        D = x1*y2 - x2*y1
        # discriminant để xác định xem có cắt vào đường tròn hay không
        # discriminant > 0 có cắt 
        # discriminant < 0 không cắt
        # discriminant = 0 tiếp tuyến
        discriminant = (lookAheadDis**2) * (dr**2) - D**2

        # Đoạn code tìm điểm cắt 
        if discriminant >= 0:
            # Tìm các điểm cắt trên đường thẳng
            sol_x1 = (D * dy + sgn(dy) * dx * np.sqrt(discriminant)) / dr**2
            sol_x2 = (D * dy - sgn(dy) * dx * np.sqrt(discriminant)) / dr**2
            sol_y1 = (- D * dx + abs(dy) * np.sqrt(discriminant)) / dr**2
            sol_y2 = (- D * dx - abs(dy) * np.sqrt(discriminant)) / dr**2

            # Đưa các điểm tìm được vào mảng và đưa nó về toạ độ thực bằng cách cộng vào các trục thực
            sol_pt1 = [sol_x1 + currentX, sol_y1 + currentY]
            sol_pt2 = [sol_x2 + currentX, sol_y2 + currentY]

            #Xác định các giá trị X Y mút giữa hai điểm trên quỷ đạo để xác định xem điểm cắt có nằm trong khoảng đó hay không
            minX = min(path[i][0], path[i+1][0])
            minY = min(path[i][1], path[i+1][1])
            maxX = max(path[i][0], path[i+1][0])
            maxY = max(path[i][1], path[i+1][1])

            # Xác định xem có điểm nào trong khoảng giữa hai điểm hay không
            # Nếu có thì kích biến intersectFound lên 1
            if (
                (minX <= sol_pt1[0] <= maxX and minY <= sol_pt1[1] <= maxY) 
                or (equalCompare(sol_pt1[0],minX) == 1 or equalCompare(sol_pt1[0],maxX) == 1 
                or equalCompare(sol_pt1[1],minY) == 1 or equalCompare(sol_pt1[1],maxY) == 1) 
                or (minX <= sol_pt2[0] <= maxX and minY <= sol_pt2[1] <= maxY) 
                or (equalCompare(sol_pt2[0],minX) == 1 or equalCompare(sol_pt2[0],maxX) == 1 
                or equalCompare(sol_pt2[1],minY) == 1 or equalCompare(sol_pt2[1],maxY) == 1)) :
                intersectFound = True 

                # TH 1 : Nếu cả hai điểm ở trong đều ở trong khoảng điểm :
                if (
                (minX <= sol_pt1[0] <= maxX and minY <= sol_pt1[1] <= maxY) 
                or (equalCompare(sol_pt1[0],minX) == 1 or equalCompare(sol_pt1[0],maxX) == 1 
                or equalCompare(sol_pt1[1],minY) == 1 or equalCompare(sol_pt1[1],maxY) == 1) 
                and (minX <= sol_pt2[0] <= maxX and minY <= sol_pt2[1] <= maxY) 
                or (equalCompare(sol_pt2[0],minX) == 1 or equalCompare(sol_pt2[0],maxX) == 1 
                or equalCompare(sol_pt2[1],minY) == 1 or equalCompare(sol_pt2[1],maxY) == 1)) :# Chúng ta sẽ kiểm tra xem điểm nào gần điểm tiếp theo cần đến trong quỷ đạo hơn :
                    if pt_to_pt_distance(sol_pt1, path[i+1]) < pt_to_pt_distance(sol_pt2, path[i+1]):
                        goalPt = sol_pt1
                    else:
                        goalPt = sol_pt2

                # TH2 : Nếu chỉ có 1 trong hai điểm nằm trong khoảng :
                else :
                    if (
                    (minX <= sol_pt1[0] <= maxX and minY <= sol_pt1[1] <= maxY) 
                    or (equalCompare(sol_pt1[0],minX) == 1 or equalCompare(sol_pt1[0],maxX) == 1 
                    or equalCompare(sol_pt1[1],minY) == 1 or equalCompare(sol_pt1[1],maxY) == 1) ):
                        goalPt = sol_pt1
                    else:
                        goalPt = sol_pt2
                
                # Tính xem là khoảng cách của setpoint tới điểm tiếp theo xem có gần hơn với vị trí robot hay không :
                # Thoát giữ index để biết là hiện tại đang ở đoạn nào 
                
                if pt_to_pt_distance (goalPt, path[i+1]) < pt_to_pt_distance ([currentX, currentY], path[i+1]):
                    lastFoundIndex = i
                    break
                else :
                    # if lastFoundIndex == path1.shape(1)-2:
                    #     goalPt = [path[lastFoundIndex+1][0], path[lastFoundIndex+1][1]]
                    # Trường hợp là vị trí setpoint xa hơn vị trí robot:
                    # index + lên một để set đến đoạn đường tiếp theo và tránh việc robot đi ngược lại
                    lastFoundIndex = i+1
            # Nếu không có điểm nào cắt :
            else :
                foundIntersection = False
                goalPt = [path[lastFoundIndex][0], path[lastFoundIndex][1]]
    
    return goalPt

In [ ]:
# Đi thẳng tới :
path0 = np.array([[0,0],[0.5,0],[0,0]])
# Đi thẳng lùi :
path1 = np.array([[0,0],[-0.5,0],[0,0]])
# Đi ngang trái :
path2 = np.array([[0,0],[0,0.5],[0,0]])
# Đi ngang phải :
path4 = np.array([[0,0],[0,-0.5],[0,0]])
# Chế độ đứng im 
pathDefault = np.array([[0,0]])

In [73]:
def firstPositonControl():
    try:
        signal.signal(signal.SIGINT, signal_handler)
        msecsElapsed = 0
        vec = [0,0,0]

        OdoOffsetX = OdometryRead()[0]
        OdoOffsetY = OdometryRead()[1]
        print(OdometryRead()[0])

        display = [0,0,0]
        pidX = PID(1,0.5,-0.5)
        pidY = PID(1,0.5,-0.5)
        pidTheta = PID(0.03,0.3,-0.3) 
        goaltheta = 0
        pathDesiried = pathDefault
        
        while False == bumper() and True == run:
            # Đọc ví trí robot từ bộ đo đường 
            OdoX = OdometryRead()[0] 
            OdoY = OdometryRead()[1] 
            OdoR = OdometryRead()[2]

            # Cập nhật vị trí robot để đưa vào tính toán 
            currentPos = [OdoX,OdoY]
            goalPt = pure_pursuit_step (pathDesired, currentPos, lookAheadDis, lastFoundIndex)


            # Đưa thông số vào bộ PID để tính toán cho robot chạy bám theo quỷ đạo 
            u = pidX.PidCal(goalPt[0],OdoX)
            v = pidY.PidCal(goalPt[1],OdoY)

            # Sử dụng ma trận xoay để robot chạy xoay trên một đường thẳng 
            uControl = (math.cos(-OdoR)*u - math.sin(-OdoR)*v)
            vControl = (math.sin(-OdoR)*u + math.cos(-OdoR)*v)
            
            # Đưa độ lớn của 3 vector tính toán được vào mảng để điều khiển robot 
            vec[0] = uControl
            vec[1] = vControl
            vec[2] = pidTheta.PidCal(goaltheta,OdoR*180/math.pi)
    

            # Hiển thị thông số để kiểm tra robot 
            # display[0] = OdoX
            # display[1] = OdoY
            # display[2] = OdoR*180/math.pi
            # print(display)
            # print(goalPt)

            # Thời gian lấy mẫu
            time.sleep(0.05)
            msecsElapsed += 50
            
            set_vel(vec)
        set_vel([0,0,0])
    except Exception as e:
        print(e)
        return 1
    return 0

In [74]:
ROBOTINOIP = "192.168.193.210"
#ROBOTINOIP = "192.168.1.223"
PARAMS = {'sid':'example_circle'}
run = True

if __name__ == "__main__":
    firstPositonControl()

0.21544832074077402
0
[0.21544832074077402, -0.0012578320513189241, -0.12000000707456393]
[0.3296027966559076, 0.3296027966559076]
0
[0.21544832074077402, -0.0012578320513189241, -0.12000000707456393]
[0.3296027966559076, 0.3296027966559076]
0
[0.21551641153369583, -0.0012186894039828973, -0.10714286447419914]
[0.3296493644471714, 0.3296493644471714]
0
[0.2168794501768812, 0.006031469678866707, 0.07857144603413739]
[0.33536567023543723, 0.33536567023543723]
0
[0.23350806009776295, 0.016959661722135914, -0.017785703604920936]
[0.3477798062566074, 0.34777980625660737]
0
[0.2697274673617764, 0.07271016389973858, -0.20000000845588656]
[0.3982563577557039, 0.3982563577557039]
0
[0.28222781227207094, 0.09920009904634342, -0.2800000198423691]
[0.4206600668414583, 0.4206600668414583]
0
[0.29392136805040703, 0.17635242830282585, -0.4000000135767198]
[0.4755415281271592, 0.4755415281271592]
0
[0.31950951226507923, 0.19522630986711276, -0.3971428484873327]
[0.4969267167911758, 0.4969267167911757]